<a href="https://colab.research.google.com/github/SamrPaul/Forecast-of-Covid-Cases-in-US-3rd-Sem-Project-BTEch-/blob/main/Forecast_of_Covid_19_using_FBProphet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stage 1**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
from datetime import datetime
%matplotlib inline
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/datasets/covid-19/main/data/time-series-19-covid-combined.csv')

In [ ]:
df.columns

Index(['Date', 'Country/Region', 'Province/State', 'Confirmed', 'Recovered',
       'Deaths'],
      dtype='object')

In [ ]:
df

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.0,0
1,2020-01-23,Afghanistan,NaN,0,0.0,0
2,2020-01-24,Afghanistan,NaN,0,0.0,0
3,2020-01-25,Afghanistan,NaN,0,0.0,0
4,2020-01-26,Afghanistan,NaN,0,0.0,0
...,...,...,...,...,...,...
231739,2022-04-12,Zimbabwe,NaN,247094,0.0,5460
231740,2022-04-13,Zimbabwe,NaN,247160,0.0,5460
231741,2022-04-14,Zimbabwe,NaN,247208,0.0,5462
231742,2022-04-15,Zimbabwe,NaN,247237,0.0,5462


In [ ]:
df.head()

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,NaN,0,0.0,0
1,2020-01-23,Afghanistan,NaN,0,0.0,0
2,2020-01-24,Afghanistan,NaN,0,0.0,0
3,2020-01-25,Afghanistan,NaN,0,0.0,0
4,2020-01-26,Afghanistan,NaN,0,0.0,0


In [ ]:
df.drop(['Province/State'],axis=1,inplace=True)

In [ ]:
df

,Date,Country/Region,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0.0,0
1,2020-01-23,Afghanistan,0,0.0,0
2,2020-01-24,Afghanistan,0,0.0,0
3,2020-01-25,Afghanistan,0,0.0,0
4,2020-01-26,Afghanistan,0,0.0,0
...,...,...,...,...,...
231739,2022-04-12,Zimbabwe,247094,0.0,5460
231740,2022-04-13,Zimbabwe,247160,0.0,5460
231741,2022-04-14,Zimbabwe,247208,0.0,5462
231742,2022-04-15,Zimbabwe,247237,0.0,5462


In [ ]:
df.describe()

,Confirmed,Recovered,Deaths
count,2.317440e+05,2.186880e+05,231744.000000
mean,5.132362e+05,1.062116e+05,9760.170231
std,3.002239e+06,8.397413e+05,49749.871513
min,0.000000e+00,0.000000e+00,0.000000
25%,2.590000e+02,0.000000e+00,2.000000
50%,5.725000e+03,5.500000e+01,73.000000
75%,1.095645e+05,5.002000e+03,1777.000000
max,8.062512e+07,3.097475e+07,988609.000000


In [ ]:
df.corr()

,Confirmed,Recovered,Deaths
Confirmed,1.000000,0.283841,0.916491
Recovered,0.283841,1.000000,0.324056
Deaths,0.916491,0.324056,1.000000


Current Cases = Confirmed Cases - Recovered Cases - Deaths

In [ ]:
df['CurrentCase'] = df['Confirmed'] - df['Recovered'] - df['Deaths']

In [ ]:
df.head()

,Date,Country/Region,Confirmed,Recovered,Deaths,CurrentCase
0,2020-01-22,Afghanistan,0,0.0,0,0.0
1,2020-01-23,Afghanistan,0,0.0,0,0.0
2,2020-01-24,Afghanistan,0,0.0,0,0.0
3,2020-01-25,Afghanistan,0,0.0,0,0.0
4,2020-01-26,Afghanistan,0,0.0,0,0.0


In [ ]:
date_df = df.groupby('Date')[['Confirmed','Recovered','Deaths','CurrentCase']].sum()

In [ ]:
date_df.head()

,Confirmed,Recovered,Deaths,CurrentCase
Date,,,,
2020-01-22,557,30.0,17,510.0
2020-01-23,657,32.0,18,605.0
2020-01-24,944,39.0,26,876.0
2020-01-25,1437,42.0,42,1350.0
2020-01-26,2120,56.0,56,2005.0


In [ ]:
date_df.tail()

,Confirmed,Recovered,Deaths,CurrentCase
Date,,,,
2022-04-12,500880363,0.0,6185040,491187741.0
2022-04-13,501920234,0.0,6189593,492159512.0
2022-04-14,502892186,0.0,6193401,493108839.0
2022-04-15,503606396,0.0,6195647,493828829.0
2022-04-16,504155459,0.0,6197159,494362715.0


#**Stage 2: Predicting using FB Prophet**

In [ ]:
!pip install prophet
!pip install yfinance


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 16.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0

In [ ]:
from prophet import Prophet

In [ ]:
import pandas as pd
full_df = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv')

HTTPError: ignored

In [ ]:
full_df

In [ ]:
full_df.head()      #FIRST FIVE DATA OF THE DATASET

In [ ]:
full_df.tail()     #FINDING LAST FEW LINES OF THE DATASET

Extracting US Data of the whole Data

In [ ]:
division = 'country'
region = 'United States'
#'ConfirmedCases' or 'ConfirmedDeaths'
prediction = 'ConfirmedCases'
full_df = pd.read_csv('https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv',
                usecols=['Date','CountryName','RegionName','Jurisdiction',
                           'ConfirmedCases','ConfirmedDeaths'],
                parse_dates=['Date'],
                encoding="ISO-8859-1",
                dtype={"RegionName": str,
                        "CountryName":str})

if division == 'country':
    df = full_df[(full_df['Jurisdiction'] == 'NAT_TOTAL') & (full_df['CountryName'] == region)][:-1]
elif division == 'state':
    df = full_df[(full_df['Jurisdiction'] == 'STATE_TOTAL') & (full_df['RegionName'] == region)][:-1]

In [ ]:
df

In [ ]:
df

**RENAMING DATASET FOR FORECASTING**

In [ ]:
cdf = df[['Date','ConfirmedCases']].rename({'Date':'ds','ConfirmedCases':'y'},axis = 'columns')
print(cdf.dtypes)

In [ ]:
cdf['ds']

In [ ]:
cdf['ds'] = df['Date'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d'))
print(cdf.dtypes)
#cdf['ds'] = datetime.strptime(df['Date'],'%y%m%d')

In [ ]:
cdf

In [ ]:
train =  cdf[(cdf['ds']>='2020-01-01')&(cdf['ds']<='2021-05-29')]

In [ ]:
test = cdf[(cdf['ds']>'2021-05-29')]

In [ ]:
print(train.shape)

In [ ]:
print(test.shape)

**PLOTTING DATA FROM FIRST WEEK OF MAY TILL DATE**

In [ ]:
from datetime import datetime
plt.figure(figsize=(25,10))
plt.ylabel("Cases")
plt.grid(True)
time=pd.date_range(start='2021-05-01',end=datetime.date(datetime.now())).tolist()
plt.xlabel("Time")
plt.plot(cdf['ds'],cdf['y'],color='b',label ='Confirmed')
plt.title('forecast of cases Over Time')
plt.legend()
plt.show()
print(datetime.date(datetime.now()))

In [ ]:
df = df[['Date',prediction]].rename(columns = {'Date':'ds', prediction:'y'})

In [ ]:
df

In [ ]:
forecast_length = 30     #30 days from now
# instantiate and fit the model
model = Prophet()
model.fit(cdf)

In [ ]:
future1 = model.make_future_dataframe(periods=forecast_length)  # make the forecast to the end of the 'future' dataframe
forecast = model.predict(future1)

In [ ]:
future1.tail()

In [ ]:
today = datetime.date(datetime.now())
today
op = print(today)

In [ ]:
td = str(today)

In [ ]:
to_plot = forecast[forecast.ds > td].merge(df, how='left')

In [ ]:
fig = model.plot_components(forecast)

****PLOTTING THE GRAPH FOR FORECASTED DATA****

In [ ]:
plt.figure(figsize = (20,8))
plt.plot(to_plot['ds'], to_plot['yhat'], label='Forecasted Cases',color='B',marker='d')
plt.fill_between(to_plot['ds'], to_plot['yhat_upper'], to_plot['yhat_lower'], alpha=.2, label='Confidence',color='m',linestyle="solid")
plt.title('FBProphet Forecasted COVID-19 cases for 30 days')
plt.legend()
plt.grid(True)
plt.savefig('prophet_forecast.png')
plt.show()
print('\n\n\t\t\t\t\t\t\t  The "forecast" DataFrame \n')

# **Stage 3**

In [ ]:
# Assuming Holidays in the US
# reference: https://www.calendarlabs.com/holidays/us/2020
holiday_dates = pd.DataFrame({
        'holiday':'2020-2021',
        'ds':pd.to_datetime(['2020-01-01','2020-01-20','2020-02-14','2020-02-17',
                             '2020-04-10','2020-04-12','2020-05-10','2020-05-25',
                             '2020-06-21','2020-07-03','2020-07-04','2020-09-07',
                             '2020-10-12','2020-10-31','2020-11-11','2020-11-26',
                             '2020-12-25',
                             '2021-01-01','2021-01-18','2021-02-14','2021-02-15',
                             '2021-04-02','2021-04-04','2021-05-09','2021-05-31',
                             '2021-06-04','2021-06-20','2021-07-04','2021-07-05',
                             '2021-09-06','2021-10-11','2021-10-31','2021-11-11',
                             '2021-11-25','2021-12-25'])
    })

In [ ]:
model2 = Prophet(interval_width=0.95, seasonality_mode='additive',
                 holidays=holiday_dates)

In [ ]:
model2.fit(train)

In [ ]:
forecast = model2.predict(future1)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig2 = model2.plot_components(forecast)

In [ ]:
model3 = Prophet(interval_width=0.95, seasonality_mode='additive',
                 holidays=holiday_dates)

In [ ]:
model3.fit(test)

In [ ]:
forecast = model3.predict(future1)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig3 = model3.plot_components(forecast)